# Count-min sketch

> Bloom filter for item counting.

CMS is an expansion on simple bloom filters to allow for item counting in data streams. Whereas bloom filter only had one vector of binay values, CMS initiates a matrix $d*w$ matrix of counters. Once an element in data stream is observed, it is hashed with $d$ distinct hash functions, exactly as with bloom filters.

In [40]:
import(
    "github.com/spaolacci/murmur3"
)

func string2hash(data string) [2]uint64 {
    return hasher([]byte(data))
}

func hasher(data []byte) [2]uint64 {
    hash := murmur3.New128()
    hash.Write(data)
    h1, h2 := hash.Sum128()
    return [2]uint64{
        h1, h2,
    }
}

// https://www.eecs.harvard.edu/~michaelm/postscripts/rsa2008.pdf
func transformHashes(h1, h2, i, size uint64) uint64 {
  return  ( h1 + i * h2 ) % size
}

Counter at each location $w_{id}$ is then incremented by 1. We can then query the data structure to return all counters for each hash functions, while only the smallest value is reported as estimated count. Logically, the item counter with smallest value has the least hash collisions and gives us the most accurate estimation of how many times that item has been seen.

> Like bloom filter, CMS can overestimate the counts. However, it can never underestimate.

In [41]:
import (
	"errors"
	"math"
	"sync/atomic"
)

// CountMinSketch is a counting BloomFilter
// depth corresponds to number of distinct hashing funcitons used
// with corresponds to number of counters in each hash set
// https://blog.demofox.org/2015/02/22/count-min-sketch-a-probabilistic-histogram/
type CountMinSketch struct {
	depth uint64
	width uint64
	count [][]uint64
}

// InitMinSketchWithEstimate instantiates a new CMS object with user defined estimate parameters
// width = [ e / epsilon ]
// depth = [ ln( 1 / delta ) ]
// hash = hashing method to use ( <= 1 for murmur, 2 for fnv, else mix of both)
func InitMinSketchWithEstimate(epsilon, delta float64) (s *CountMinSketch, err error) {
	depth, width := estimateCountMinSize(epsilon, delta)
	if epsilon <= 0 || epsilon >= 1 {
		return nil, errors.New("CountMinSketch.Init: epsilon must be 0 < eps < 1")
	}
	if delta <= 0 || delta >= 1 {
		return nil, errors.New("CountMinSketch.Init: delta must be 0 < eps < 1")
	}
	s = &CountMinSketch{
		depth: depth,
		width: width,
	}
	s.count = make([][]uint64, depth)
	for i := uint64(0); i < depth; i++ {
		s.count[i] = make([]uint64, width)
	}
	return s, err
}

func estimateCountMinSize(epsilon, delta float64) (depth, width uint64) {
	depth = uint64(math.Ceil(math.Log(1.0 / delta)))
	width = uint64(math.Ceil(math.E / epsilon))
	return
}

func (s *CountMinSketch) genLocs(data []byte) (locations []uint64) {
	locations = make([]uint64, s.depth)
	h := hasher(data)
	for i := uint64(0); i < uint64(s.depth); i++ {
		locations[i] = transformHashes(h[0], h[1], i, uint64(s.width))
	}
	return
}

// Increment item count in CMS without returning the new estimated value
func (s *CountMinSketch) Increment(data []byte) *CountMinSketch {
	// location = hashing function i < depth
	for i, elem := range s.genLocs(data) {
		atomic.AddUint64(&s.count[i][elem], 1)
	}
	return s
}

// IncrementGetVal is a combination of Increment() and QueryMin() methods that returns new estimation upon adding each element
// deduplicates needed work if estimation has to be compared to threshold
func (s *CountMinSketch) IncrementGetVal(data []byte) (min uint64) {
	// location = hashing function i < depth
	for i, elem := range s.genLocs(data) {
		c := &s.count[i][elem]
		atomic.AddUint64(c, 1)
		if min == 0 || *c < min {
			min = *c
		}
	}
	return
}

// IncrementStringGetVal converts textual input before returning IncrementGetVal()
func (s *CountMinSketch) IncrementStringGetVal(data string) (min uint64) {
	return s.IncrementGetVal([]byte(data))
}

// IncrementString converts textual input before returning Increment()
func (s *CountMinSketch) IncrementString(data string) *CountMinSketch {
	return s.Increment([]byte(data))
}

// QueryMin returns estimated value for item
// smallest count = least collisions, thus most accurate estimation
// if smallest value is zero, the item has not been counted before.
// CMS cannot under-estimate by definition, thus any subsequent checks are waste of CPU cycles
func (s *CountMinSketch) QueryMin(data []byte) (min uint64) {
	for i, elem := range s.genLocs(data) {
		c := s.count[i][elem]
		if c == 1 {
			min = 1
			break
		} else if min == 0 || c < min {
			min = c
		}
	}
	return
}

// QueryString converts textual input before returning Query()
func (s *CountMinSketch) QueryString(data string) uint64 {
	return s.QueryMin([]byte(data))
}

In [42]:
import(
    "fmt"
)

epsilon := 0.01
delta := 0.01

cms, err := InitMinSketchWithEstimate(epsilon, delta)

In [43]:
fmt.Println("Initiated CMS with", cms.depth, "hash functions and", cms.width, "counters per function.")

Initiated CMS with 5 hash functions and 272 counters per function.
67
<nil>


In [44]:
var count uint64
for i := 0; i < 10; i++ {
    count = cms.IncrementStringGetVal("aaa")
}
for i := 0; i < 2; i++ {
    count = cms.IncrementStringGetVal("aab")
}

Again, like with Bloom filters, query is a one-way operation. I.e., you can only query items that you know about.

In [45]:
count := cms.QueryString("aaa")
fmt.Println(count)

10
3
<nil>


In [46]:
count := cms.QueryString("aab")
fmt.Println(count)

2
2
<nil>


In [47]:
count := cms.QueryString("aac")
fmt.Println(count)

0
2
<nil>
